In [1]:
import cv2
import numpy as np
import os
import random
from tqdm import tqdm

In [2]:
def get_corners_from_yolo(bboxes, img_w, img_h):
    """Convert YOLO format [class, x_center, y_center, width, height] to pixel corner format [x1, y1, x2, y2]."""
    corners = []
    for bbox in bboxes:
        _, x_center, y_center, width, height = bbox
        w = width * img_w
        h = height * img_h
        x1 = (x_center * img_w) - (w / 2)
        y1 = (y_center * img_h) - (h / 2)
        x2 = x1 + w
        y2 = y1 + h
        corners.append([x1, y1, x2, y2])
    return np.array(corners)

In [3]:
def get_yolo_from_corners(corners, class_ids, img_w, img_h):
    """Convert pixel corner format [x1, y1, x2, y2] to YOLO format."""
    yolo_boxes = []
    for i, corner in enumerate(corners):
        x1, y1, x2, y2 = corner
        x1, x2 = min(x1, x2), max(x1, x2)
        y1, y2 = min(y1, y2), max(y1, y2)
        
        x1, y1 = max(0, x1), max(0, y1)
        x2, y2 = min(img_w, x2), min(img_h, y2)

        if x2 <= x1 or y2 <= y1:
            continue

        width = x2 - x1
        height = y2 - y1
        x_center = (x1 + width / 2) / img_w
        y_center = (y1 + height / 2) / img_h
        
        yolo_boxes.append([
            class_ids[i], 
            np.clip(x_center, 0, 1), 
            np.clip(y_center, 0, 1), 
            np.clip(width / img_w, 0, 1), 
            np.clip(height / img_h, 0, 1)
        ])
    return np.array(yolo_boxes)

In [4]:
def rotate_boxes(corners_4p, angle, cx, cy):
    """
    Rotate the 4 corners of each bounding box and return the new enclosing box.
    """
    all_corners = corners_4p.reshape(-1, 2)
    all_corners_hom = np.hstack((all_corners, np.ones((all_corners.shape[0], 1))))

    M = cv2.getRotationMatrix2D((cx, cy), angle, 1.0)

    rotated_all_corners = np.dot(M, all_corners_hom.T).T
    rotated_corners_4p = rotated_all_corners.reshape(-1, 4, 2)

    x_min = np.min(rotated_corners_4p[:, :, 0], axis=1)
    y_min = np.min(rotated_corners_4p[:, :, 1], axis=1)
    x_max = np.max(rotated_corners_4p[:, :, 0], axis=1)
    y_max = np.max(rotated_corners_4p[:, :, 1], axis=1)

    return np.column_stack((x_min, y_min, x_max, y_max))

In [5]:
def augment_image(image, bboxes):
    h, w = image.shape[:2]
    class_ids = bboxes[:, 0].copy()
    augmentations = {}
    
    # 1. Vertical Flip
    img_vf = cv2.flip(image, 0)
    bboxes_vf = bboxes.copy()
    bboxes_vf[:, 2] = 1 - bboxes_vf[:, 2] # Flip y-coordinate
    augmentations['vf'] = (img_vf, bboxes_vf)

    # 2. Brightness Adjustment
    brightness = random.uniform(0.6, 1.4)
    img_bright = cv2.convertScaleAbs(image, alpha=brightness, beta=0)
    augmentations['bright'] = (img_bright, bboxes.copy())
    
    # 3. Rotation
    angle = random.uniform(-15, 15)
    rot_mat = cv2.getRotationMatrix2D((w/2, h/2), angle, 1.0)
    img_rot = cv2.warpAffine(image, rot_mat, (w, h), borderMode=cv2.BORDER_REPLICATE)
    
    pixel_corners_2p = get_corners_from_yolo(bboxes, w, h)
    if pixel_corners_2p.size == 0:
        return augmentations

    x1 = pixel_corners_2p[:, 0:1]
    y1 = pixel_corners_2p[:, 1:2]
    x2 = pixel_corners_2p[:, 2:3]
    y2 = pixel_corners_2p[:, 3:4]
    
    corners_4p = np.hstack([x1, y1, x2, y1, x1, y2, x2, y2]).reshape(-1, 4, 2)

    rotated_pixel_corners = rotate_boxes(corners_4p, angle, w/2, h/2)
    
    bboxes_rot = get_yolo_from_corners(rotated_pixel_corners, class_ids, w, h)
    if bboxes_rot.size > 0:
        augmentations['rot'] = (img_rot, bboxes_rot)

    return augmentations

In [6]:
def main():
    source_image_dir = r"C:\Empty_Parking\bounding_box\P_test"
    source_label_dir = r"C:\Empty_Parking\bounding_box\P_test_labels"
    
    dest_image_dir = r"C:\Empty_Parking\PKLot.v2-640.yolov8\train\images"
    dest_label_dir = r"C:\Empty_Parking\PKLot.v2-640.yolov8\train\labels"

    os.makedirs(dest_image_dir, exist_ok=True)
    os.makedirs(dest_label_dir, exist_ok=True)

    image_files = [f for f in os.listdir(source_image_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]
    print(f'Found {len(image_files)} original images to augment and add to the training set.')

    for image_name in tqdm(image_files, desc='Augmenting Images'):
        base_name, ext = os.path.splitext(image_name)
        label_name = f'{base_name}.txt'
        image_path = os.path.join(source_image_dir, image_name)
        label_path = os.path.join(source_label_dir, label_name)

        if not os.path.exists(label_path):
            continue
            
        image = cv2.imread(image_path)
        if image is None:
            continue
            
        with open(label_path, 'r') as f:
            try:
                bboxes = np.array([line.strip().split() for line in f.readlines()], dtype=np.float32)
            except (ValueError, IndexError):
                continue
            
        if bboxes.ndim == 1: # Handle single line label files
            bboxes = np.array([bboxes])

        if bboxes.size == 0 or bboxes.shape[1] != 5:
            continue
            
        augmented_data = augment_image(image, bboxes)
        
        for aug_type, (aug_img, aug_bboxes) in augmented_data.items():
            if aug_bboxes.size == 0:
                continue
            
            # Save to the destination training folder
            new_base_name = f'{base_name}_{aug_type}'
            aug_img_name = f'{new_base_name}{ext}'
            aug_label_name = f'{new_base_name}.txt'
            
            cv2.imwrite(os.path.join(dest_image_dir, aug_img_name), aug_img)
            np.savetxt(os.path.join(dest_label_dir, aug_label_name), aug_bboxes, fmt='%.6f')

    print('\nData augmentation and integration complete!')
    print(f'Total images in training set now: {len(os.listdir(dest_image_dir))}')

In [7]:
if __name__ == '__main__':
    main()


Found 439 original images to augment and add to the training set.


Augmenting Images: 100%|█████████████████████████████████████████████████████████████| 439/439 [00:04<00:00, 96.12it/s]



Data augmentation and integration complete!
Total images in training set now: 34988
